# Création de prompts pour le nommage et la description des clusters

In [3]:
Sys.setlocale("LC_ALL", "fr_CA.UTF-8")

# Library
library(dplyr)
library(tidyr)
library(ggplot2)
library(ggtext)
library(tidytext)
library(ellipsellm)

# Load Data and Clustering Results
kmeans_result <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_kmeans.rds") # nolint
dfUsedForClustering <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_2025.rds")
df_clusters_composition <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_composition.rds")

[1] "fr_CA.UTF-8/fr_CA.UTF-8/fr_CA.UTF-8/C/fr_CA.UTF-8/C"

In [4]:
cluster_names <- unique(dfUsedForClustering$cluster_name)
cluster_names <- cluster_names[order(cluster_names)]

prompts <- list()

for (cluster_value in cluster_names) {
  cluster_data <- dfUsedForClustering %>% filter(cluster_name == cluster_value)
  cluster_vars <- df_clusters_composition %>% filter(cluster_name == cluster_value)

  # Calculer quelques statistiques démographiques pour le cluster
  perc_hommes     <- mean(cluster_data$ses_genderMale, na.rm = TRUE) * 100
  perc_immigrants <- mean(cluster_data$ses_immigrant, na.rm = TRUE) * 100
  perc_langEn     <- mean(cluster_data$ses_languageEnglish, na.rm = TRUE) * 100
  perc_langFr     <- mean(cluster_data$ses_languageFrench, na.rm = TRUE) * 100
  perc_ses_languageOther <- mean(cluster_data$ses_languageOther, na.rm = TRUE) * 100
  perc_ethn_white <- mean(cluster_data$ses_ethnicityWhite, na.rm = TRUE) * 100
  perc_hetero     <- mean(cluster_data$ses_sexOrientationHetero, na.rm = TRUE) * 100
  educ_mean_BHS       <- mean(cluster_data$ses_educBHS, na.rm = TRUE) * 100
  educ_mean_PostHS       <- mean(cluster_data$ses_educPostHS, na.rm = TRUE) * 100
  educ_mean_Univ       <- mean(cluster_data$ses_educUniv, na.rm = TRUE) * 100
  age_mean        <- mean(cluster_data$ses_age, na.rm = TRUE) * 80
  revenu_moyen_Low    <- mean(cluster_data$ses_incomeLow, na.rm = TRUE) * 100
  revenu_moyen_Mid    <- mean(cluster_data$ses_incomeMid, na.rm = TRUE) * 100
  revenu_moyen_High    <- mean(cluster_data$ses_incomeHigh, na.rm = TRUE) * 100

  # Génération du prompt de suggestion de noms
  prompt_description <- (paste0(
  "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona ", cluster_value, " :\n\n",
  
  "Variables avec une moyenne significativement plus élevée que la moyenne de la population en général :\n",
  if (any(cluster_vars$color == "green")) {
    paste(cluster_vars[cluster_vars$color == "green", "variable"], collapse = "\n")
  } else {
    "Aucune variable n'a une moyenne significativement plus élevée que la moyenne de la population en général."
  },
  
  "\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\n",
  if (any(cluster_vars$color == "red")) {
    paste(cluster_vars[cluster_vars$color == "red", "variable"], collapse = "\n")
  } else {
    "Aucune variable avec une moyenne significativement plus basse que la moyenne de la population en général."
  },
  
  "\n\nCe cluster est composé de :\n", 
  "Hommes: ", round(perc_hommes, 1), "%, ",
  "Immigrants: ", round(perc_immigrants, 1), "%\n",
  "Âge: ", round(age_mean, 1), " ans\n",
  "Revenus:\n",
  "  Revenu Bas: ", round(revenu_moyen_Low, 1), " %\n",
  "  Revenu Mid: ", round(revenu_moyen_Mid, 1), " %\n",
  "  Revenu High: ", round(revenu_moyen_High, 1), " %\n",
  "Langues:\n",
  "  En: ", round(perc_langEn, 1), "%, ",
  "Fr: ", round(perc_langFr, 1), "%, ",
  "Autres: ", round(perc_ses_languageOther, 1), "%\n",
  "EducBHS: ", round(educ_mean_BHS, 1), "%\n",
  "EducPostHS: ", round(educ_mean_PostHS, 1), "%\n",
  "EducUniv: ", round(educ_mean_Univ, 1), "%\n",
  "Ethnie (Blanc): ", round(perc_ethn_white, 1), "%\n",
  "Orientation (Hétéro): ", round(perc_hetero, 1), "%\n"
))

  prompts[[cluster_value]] <- prompt_description
}


In [5]:
prompts

$`1`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 1 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"lifestyle_consClothesFrip\", \"lifestyle_consCoffeeIndependent\", \"lifestyle_typeTransportNoCar\", \"ses_educUniv\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"lifestyle_consCoffeeNone\", \"lifestyle_consCoffeeTimHortons\", \"lifestyle_goHuntingFreq_numeric\", \"lifestyle_smokeFreq\", \"ses_educBHS\")\n\nCe cluster est composé de :\nHommes: 52.9%, Immigrants: 14.7%\nÂge: 42.9 ans\nRevenus:\n  Revenu Bas: 11.8 %\n  Revenu Mid: 69.1 %\n  Revenu High: 19.1 %\nLangues:\n  En: 60.3%, Fr: 38.2%, Autres: 1.5%\nEducBHS: 4.4%\nEducPostHS: 20.6%\nEducUniv: 75%\nEthnie (Blanc): 88.2%\nOrientation (Hétéro): 91.2%\n"

$`2`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 2 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"lifestyle_exerciseYoga\", \"ses_dwellingApp\", \"ses_immigrant\", \"ses_incomeHigh\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"lifestyle_exerciseGym\", \"lifestyle_smokeFreq\")\n\nCe cluster est composé de :\nHommes: 27.3%, Immigrants: 40.9%\nÂge: 40 ans\nRevenus:\n  Revenu Bas: 4.5 %\n  Revenu Mid: 59.1 %\n  Revenu High: 36.4 %\nLangues:\n  En: 72.7%, Fr: 27.3%, Autres: 0%\nEducBHS: 13.6%\nEducPostHS: 22.7%\nEducUniv: 63.6%\nEthnie (Blanc): 63.6%\nOrientation (Hétéro): 86.4%\n"

$`3`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 3 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"lifestyle_consCoffeeTimHortons\", \"lifestyle_exerciseGym\", \"lifestyle_goHuntingFreq_numeric\", \"lifestyle_hasTattoos\", \"lifestyle_ownPetDog\", \"lifestyle_smokeFreq\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"lifestyle_consCoffeeIndependent\", \"lifestyle_consCoffeeNone\", \"lifestyle_favAlcoolDontDrink\", \"lifestyle_ownPetNone\", \"ses_educBHS\", \"ses_languageFrench\")\n\nCe cluster est composé de :\nHommes: 71.9%, Immigrants: 16.5%\nÂge: 26.5 ans\nRevenus:\n  Revenu Bas: 12.2 %\n  Revenu Mid: 73.4 %\n  Revenu High: 14.4 %\nLangues:\n  En: 76.3%, Fr: 20.9%, Autres: 2.9%\nEducBHS: 12.2%\nEducPostHS: 44.6%\nEducUniv: 43.2%\nEthnie (Blanc): 67.6%\nOrientation (Hétéro): 85.6%\n"

$`4`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 4 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"lifestyle_consCoffeeTimHortons\", \"ses_educUniv\", \"ses_immigrant\", \"ses_incomeHigh\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"lifestyle_consClothesFrip\", \"lifestyle_consCoffeeIndependent\", \"lifestyle_consCoffeeNone\", \"lifestyle_goHuntingFreq_numeric\", \"lifestyle_smokeFreq\", \"ses_educBHS\", \"ses_languageFrench\")\n\nCe cluster est composé de :\nHommes: 52%, Immigrants: 25%\nÂge: 42.4 ans\nRevenus:\n  Revenu Bas: 9.8 %\n  Revenu Mid: 68.2 %\n  Revenu High: 22 %\nLangues:\n  En: 74.7%, Fr: 19.9%, Autres: 5.4%\nEducBHS: 0.3%\nEducPostHS: 36.5%\nEducUniv: 63.2%\nEthnie (Blanc): 73%\nOrientation (Hétéro): 84.8%\n"

$`5`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 5 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"lifestyle_consCoffeeNone\", \"lifestyle_ownPetNone\", \"ses_dwellingDetachedHouse\", \"ses_educUniv\", \"ses_languageFrench\")\n\nVariables avec une moyenne significativemen

In [6]:
reponses_openai <- list()
# Afficher ou traiter tous les prompts
for (cluster in names(prompts)) {
  # Par exemple, envoyer chaque prompt à OpenAI (si la fonction est correctement définie)
  conv <- openai_create_conversation(prompts[[cluster]])
  resp <- openai_chat_completion(conv)
  reponses_openai[[cluster]] <- resp$content
}


In [7]:
reponses_openai

$`1`
[1] "Pour une personne vivant au Canada ayant les caractéristiques décrites, je propose le prénom \"Julien\". Ce prénom est utilisé dans les communautés anglophones et francophones du Canada, ce qui correspond bien au pourcentage de personnes parlant anglais et français dans ce groupe.\n\n**Persona 1 - Julien :**\n\nJulien est un homme de 43 ans vivant dans une grande ville canadienne. Il a obtenu un diplôme universitaire et appartient à la classe moyenne avec des revenus moyens. Julien préfère un mode de vie durable et responsable. Il aime acheter des vêtements de seconde main et fréquente régulièrement des cafés indépendants pour savourer des cafés de qualité. \n\nIl n'utilise pas de voiture, préférant des moyens de transport plus écologiques comme le vélo ou les transports en commun. Ce choix reflète son engagement envers l'environnement et un style de vie urbain. Bien qu'il soit majoritairement anglophone, Julien parle couramment le français, ce qui enrichit ses interactions culturelles.\n\nJulien ne fume pas et n'est pas intéressé par la chasse, deux activités peu courantes parmi ceux qui partagent son style de vie. En tant qu'homme blanc hétérosexuel, il s'intègre dans la majorité socio-culturelle du pays, tout en représentant une minorité en termes de consommation et de mode de vie. Julien est un exemple d'équilibre entre engagement social et réussite individuelle, vivant pleinement ses valeurs dans son quotidien."

$`2`
[1] "Pour une personne habitant au Canada dans ce profil, je propose le prénom \"Émilie\" pour une femme ou \"Marc\" pour un homme. \n\n### Persona 2 : Marc/Émilie\n\n- **Prénom** : Marc (pour un homme) / Émilie (pour une femme)\n- **Âge** : 40 ans\n- **Sexe** : Probablement féminin (72.7%)\n- **Statut d'immigration** : Immigrant (40.9%)\n- **Langue** : Principalement anglophone (72.7%), avec une part francophone (27.3%)\n- **Revenus** : Plutôt élevé, avec une proportion significative dans la catégorie de revenu élevé (36.4%)\n- **Habitation** : Probablement un appartement (ses_dwellingApp)\n- **Niveau d'éducation** : Principalement universitaire (63.6%)\n- **Ethnie** : Majoritairement blanc (63.6%)\n- **Orientation sexuelle** : Hétérosexuel (86.4%)\n- **Mode de vie** :\n  - Pratique régulière du yoga (lifestyle_exerciseYoga)\n  - Moins d'exercice en salle de sport (lifestyle_exerciseGym)\n  - Ne fume pas ou fume peu fréquemment (lifestyle_smokeFreq)\n\n### Description :\n\nMarc/Émilie est une personne bien éduquée vivant au Canada, probablement dans une grande ville où les communautés d'immigrants sont plus importantes. Étant un(e) immigrant(e) ou ayant un bagage culturel diversifié, il/elle intègre facilement des pratiques telles que le yoga dans son quotidien pour maintenir un mode de vie sain. Il/elle privilégie la vie en appartement, peut-être pour sa praticité et son emplacement central. Sa carrière est probablement bien établie, avec des revenus confortables qui lui permettent de profiter de loisirs comme le yoga. Il/elle parle principalement anglais mais peut aussi avoir une certaine connaissance du français, ce qui est un atout dans certains secteurs professionnels au Canada. Marc/Émilie est conscient(e) de sa santé et évite les habitudes nuisibles comme le tabagisme."

$`3`
[1] "Pour une personne habitant au Canada correspondant au persona 3 décrit, je propose le prénom \"Brandon\". Ce prénom est assez courant au Canada et peut convenir à un homme jeune et dynamique. Voici une brève description de Brandon :\n\n**Brandon, 26 ans**  \nBrandon est un jeune homme énergique et sociable qui vit au Canada. Il est passionné par le sport et fréquente régulièrement le gymnase pour rester en forme. En plus de sa routine d'exercice, Brandon apprécie les activités en plein air comme la chasse. Il est amateur de café et préfère les chaînes de café bien établies comme Tim Hortons, plutôt que les cafés indépendants.\n\nBrandon a un style de vie plutôt décontracté, arborant plusieurs tatouages qui reflète

In [12]:
description_clusters <- data.frame(prompt = unlist(prompts), description = unlist(reponses_openai))

In [13]:
description_clusters

,prompt,description
,<chr>,<chr>
1,"Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 1 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""lifestyle_consClothesFrip"", ""lifestyle_consCoffeeIndependent"", ""lifestyle_typeTransportNoCar"", ""ses_educUniv"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""lifestyle_consCoffeeNone"", ""lifestyle_consCoffeeTimHortons"", ""lifestyle_goHuntingFreq_numeric"", ""lifestyle_smokeFreq"", ""ses_educBHS"") Ce cluster est composé de : Hommes: 52.9%, Immigrants: 14.7% Âge: 42.9 ans Revenus: Revenu Bas: 11.8 % Revenu Mid: 69.1 % Revenu High: 19.1 % Langues: En: 60.3%, Fr: 38.2%, Autres: 1.5% EducBHS: 4.4% EducPostHS: 20.6% EducUniv: 75% Ethnie (Blanc): 88.2% Orientation (Hétéro): 91.2%","Pour une personne vivant au Canada ayant les caractéristiques décrites, je propose le prénom ""Julien"". Ce prénom est utilisé dans les communautés anglophones et francophones du Canada, ce qui correspond bien au pourcentage de personnes parlant anglais et français dans ce groupe. **Persona 1 - Julien :** Julien est un homme de 43 ans vivant dans une grande ville canadienne. Il a obtenu un diplôme universitaire et appartient à la classe moyenne avec des revenus moyens. Julien préfère un mode de vie durable et responsable. Il aime acheter des vêtements de seconde main et fréquente régulièrement des cafés indépendants pour savourer des cafés de qualité. Il n'utilise pas de voiture, préférant des moyens de transport plus écologiques comme le vélo ou les transports en commun. Ce choix reflète son engagement envers l'environnement et un style de vie urbain. Bien qu'il soit majoritairement anglophone, Julien parle couramment le français, ce qui enrichit ses interactions culturelles. Julien ne fume pas et n'est pas intéressé par la chasse, deux activités peu courantes parmi ceux qui partagent son style de vie. En tant qu'homme blanc hétérosexuel, il s'intègre dans la majorité socio-culturelle du pays, tout en représentant une minorité en termes de consommation et de mode de vie. Julien est un exemple d'équilibre entre engagement social et réussite individuelle, vivant pleinement ses valeurs dans son quotidien."
2,"Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 2 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""lifestyle_exerciseYoga"", ""ses_dwellingApp"", ""ses_immigrant"", ""ses_incomeHigh"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""lifestyle_exerciseGym"", ""lifestyle_smokeFreq"") Ce cluster est composé de : Hommes: 27.3%, Immigrants: 40.9% Âge: 40 ans Revenus: Revenu Bas: 4.5 % Revenu Mid: 59.1 % Revenu High: 36.4 % Langues: En: 72.7%, Fr: 27.3%, Autres: 0% EducBHS: 13.6% EducPostHS: 22.7% EducUniv: 63.6% Ethnie (Blanc): 63.6% Orientation (Hétéro): 86.4%","Pour une personne habitant au Canada dans ce profil, je propose le prénom ""Émilie"" pour une femme ou ""Marc"" pour un homme. ### Persona 2 : Marc/Émilie - **Prénom** : Marc (pour un homme) / Émilie (pour une femme) - **Âge** : 40 ans - **Sexe** : Probablement féminin (72.7%) - **Statut d'immigration** : Immigrant (40.9%) - **Langue** : Principalement anglophone (72.7%), avec une part francophone (27.3%) - **Revenus** : Plutôt élevé, avec une proportion significative dans la catégorie de revenu élevé (36.4%) - **Habitation** : Probablement un appartement (ses_dwellingApp) - **Niveau d'éducation** : Principalement universitaire (63.6%) - **Ethnie** : Majoritairement blanc (63.6%) - **Orientation sexuelle** : Hétérosexuel (86.4%) - **Mode de vie** : - Pratique régulière du yoga (lifestyle_exerciseYoga) - Moins d'exercice en salle de sport (lifestyle_exerciseGym) - Ne fume pas ou fume peu fréquemment (lifestyle_smokeFreq) ##

In [14]:
write.csv(description_clusters, "description_clusters.csv", row.names = FALSE)

In [15]:
saveRDS(description_clusters, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_description.rds")